<a href="https://colab.research.google.com/github/mu06905/GPU-Accelerated-Programming-in-Cuda-2023/blob/main/Week6/DotProductReduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explanation:
We have two arrays a and b. tid (global thread index) would range from [0, 33972].
We have 256 threads per bloclks and 32 blocks.
In the first loop, each thread of each block multiplies an element of a and b and adds them together with an offset of GridDim x blockDim, which is (32*256 = 8198). So for the first block we have:

First iteration: tid = 0:

temp + =a[0] x b[0] (block 0 , thread 0)

temp += a[1] x b[1] (block 0, thread 1)

...

temp += a[256] x b[256] (block 0, thread 256)

Second Iteration: tid = 8196:



then tid += 8196, which gives:

temp += a[8196] x b[8196] (block 0, thread 0)

temp += a[8197] x b[8197] (block 0, thread 1)

...

Note that GridDim x BlockDim != N

Now for each block we have a cache 32 elements each which conataining partial sum of the array. cache1 contains sum of 
[a[0] * b[0], a[1] x b[1] ... ,a[256] x b[256], a[8196] x b [8196] , a[8197] x b[8197] ...]

 cache2 contains sum of 
[a[257] * b[257], a[258] x b[258] ... ,a[512] x b[512], a[8708] x b [8708] , a[8709] x b[8709] ...]

So we have 32 cahes, we use reduction sum to sum over these cache arrays and store them in the first element of the cache of each block.

which is then stored inside another array and the final sum is calculated on cpu



In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

In [6]:
%%cu 
#include <stdio.h>
#include <cuda.h>

const int N = 33*1024;
const int blocksPerGrid = 32;
const int threadsPerBlock = 256; //multiple of 2 for reduction code
const int size = N * sizeof(int);

__global__ void dot(int* a, int*b, int*c, int N){
    __shared__ int cache[threadsPerBlock];
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int cache_index = threadIdx.x;
    int temp = 0;
    while(tid<N){
        temp += a[tid] * b[tid];
        tid += blocksPerGrid * threadsPerBlock;
    }
    cache[cache_index] = temp;
    __syncthreads();

    int i = blockDim.x / 2;
    while(i != 0){
        if (cache_index < i){
            cache[cache_index] += cache[cache_index + i];
        }
        __syncthreads();
        i/=2;
    }
    if(cache_index == 0)
    c[blockIdx.x] = cache[0];
}

int main(){
    int* h_a = (int*)malloc(size);
    int* h_b = (int*)malloc(size);
    int* h_c = (int*)malloc(size);

    int* d_a;
    int *d_b;
    int* d_c;

    cudaMalloc((void**)&d_a, size);
    cudaMalloc((void**)&d_b, size);
    cudaMalloc((void**)&d_c, size);

    for(int i = 0; i<N; i++){
        h_a[i] = i * 2;
        h_b[i] = i;
        h_c[i] = 0;
    }

    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_c, h_c, size, cudaMemcpyHostToDevice);

    dot<<<blocksPerGrid, threadsPerBlock>>>(d_a,d_b,d_c,N);

    cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);

    int sum = 0;
    int j = 0;
    for(int i = 0; i<threadsPerBlock; i++){
        sum += h_c[i];
        j++;
    }

    printf("%d\n", sum);
    printf("%d,%d",N*(N+1)/2,j);

    cudaFree(&d_a);
    cudaFree(&d_b);
    cudaFree(&d_c);

    free(h_a);
    free(h_b);
    free(h_c);

    return 0;
}


1005595648
570966528,256
